### Problem Description 

* population = set of n individuals 
* an individual = one valid fantasy league
* Each league consists of 5 teams
* Each team is itself a list of 7 player dictionaries

<br> List of Individuals → Each Individual = List of 5 Teams → Each Team = List of 7 Players → Each Player = Dictionary

 Constraints:
 * every player must be assigned to exactly one team
 * each of the 5 teams must have 1 GK, 2 DEF, 2 MID, 2 FWD
 * the total salary of any team cannot exceed €750 million
 * all teams must be formed (no incompete teams nor not assigned players) 

In [2]:
from initial_pop import load_players_from_csv, create_initial_population

In [3]:
players = load_players_from_csv("data/players(in).csv")
players

[{'name': 'Alex Carter', 'position': 'GK', 'skill': 85, 'cost': 90.0},
 {'name': 'Jordan Smith', 'position': 'GK', 'skill': 88, 'cost': 100.0},
 {'name': 'Ryan Mitchell', 'position': 'GK', 'skill': 83, 'cost': 85.0},
 {'name': 'Chris Thompson', 'position': 'GK', 'skill': 80, 'cost': 80.0},
 {'name': 'Blake Henderson', 'position': 'GK', 'skill': 87, 'cost': 95.0},
 {'name': 'Daniel Foster', 'position': 'DEF', 'skill': 90, 'cost': 110.0},
 {'name': 'Lucas Bennett', 'position': 'DEF', 'skill': 85, 'cost': 90.0},
 {'name': 'Owen Parker', 'position': 'DEF', 'skill': 88, 'cost': 100.0},
 {'name': 'Ethan Howard', 'position': 'DEF', 'skill': 80, 'cost': 70.0},
 {'name': 'Mason Reed', 'position': 'DEF', 'skill': 82, 'cost': 75.0},
 {'name': 'Logan Brooks', 'position': 'DEF', 'skill': 86, 'cost': 95.0},
 {'name': 'Caleb Fisher', 'position': 'DEF', 'skill': 84, 'cost': 85.0},
 {'name': 'Nathan Wright', 'position': 'MID', 'skill': 92, 'cost': 120.0},
 {'name': 'Connor Hayes', 'position': 'MID', 's

We have to decide later what population size we want to set. By default i set 50.

In [4]:
initial_population = create_initial_population(players,population_size=50, salary_cap=750)
initial_population

[[[{'name': 'Ryan Mitchell', 'position': 'GK', 'skill': 83, 'cost': 85.0},
   {'name': 'Mason Reed', 'position': 'DEF', 'skill': 82, 'cost': 75.0},
   {'name': 'Jaxon Griffin', 'position': 'DEF', 'skill': 79, 'cost': 65.0},
   {'name': 'Hunter Cooper', 'position': 'MID', 'skill': 83, 'cost': 85.0},
   {'name': 'Connor Hayes', 'position': 'MID', 'skill': 89, 'cost': 105.0},
   {'name': 'Landon Powell', 'position': 'FWD', 'skill': 89, 'cost': 110.0},
   {'name': 'Elijah Sanders', 'position': 'FWD', 'skill': 93, 'cost': 140.0}],
  [{'name': 'Alex Carter', 'position': 'GK', 'skill': 85, 'cost': 90.0},
   {'name': 'Logan Brooks', 'position': 'DEF', 'skill': 86, 'cost': 95.0},
   {'name': 'Lucas Bennett', 'position': 'DEF', 'skill': 85, 'cost': 90.0},
   {'name': 'Austin Torres', 'position': 'MID', 'skill': 82, 'cost': 80.0},
   {'name': 'Ashton Phillips', 'position': 'MID', 'skill': 90, 'cost': 110.0},
   {'name': 'Chase Murphy', 'position': 'FWD', 'skill': 86, 'cost': 95.0},
   {'name': 'X

In [5]:
for team_idx, team in enumerate(initial_population[0], start=1):
    print(f"\nTeam {team_idx}")
    for player in team:
        print(f"{player['name']}, {player['position']}, Skill: {player['skill']}, Salary: €{player['cost']}M")
        
    team_cost = sum(player['cost'] for player in team)
    print(f"Total Team Salary: €{team_cost}M")



Team 1
Ryan Mitchell, GK, Skill: 83, Salary: €85.0M
Mason Reed, DEF, Skill: 82, Salary: €75.0M
Jaxon Griffin, DEF, Skill: 79, Salary: €65.0M
Hunter Cooper, MID, Skill: 83, Salary: €85.0M
Connor Hayes, MID, Skill: 89, Salary: €105.0M
Landon Powell, FWD, Skill: 89, Salary: €110.0M
Elijah Sanders, FWD, Skill: 93, Salary: €140.0M
Total Team Salary: €665.0M

Team 2
Alex Carter, GK, Skill: 85, Salary: €90.0M
Logan Brooks, DEF, Skill: 86, Salary: €95.0M
Lucas Bennett, DEF, Skill: 85, Salary: €90.0M
Austin Torres, MID, Skill: 82, Salary: €80.0M
Ashton Phillips, MID, Skill: 90, Salary: €110.0M
Chase Murphy, FWD, Skill: 86, Salary: €95.0M
Xavier Bryant, FWD, Skill: 90, Salary: €120.0M
Total Team Salary: €680.0M

Team 3
Jordan Smith, GK, Skill: 88, Salary: €100.0M
Ethan Howard, DEF, Skill: 80, Salary: €70.0M
Maxwell Flores, DEF, Skill: 81, Salary: €72.0M
Nathan Wright, MID, Skill: 92, Salary: €120.0M
Dominic Bell, MID, Skill: 86, Salary: €95.0M
Tyler Jenkins, FWD, Skill: 80, Salary: €70.0M
Sebas